# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
pip install dotenv

In [12]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('')

In [13]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [14]:
#Definition of the tables.
import pandas as pd

import pandas as pd

data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, department, position, etc.',
        'Salary details for each year, including bonuses and deductions.',
        'Educational studies, name of the institution, type of studies, level of degree.'
    ]
}

df = pd.DataFrame(data)

print(df)


       table                                         definition
0  employees  Employee information, name, department, positi...
1     salary  Salary details for each year, including bonuse...
2    studies  Educational studies, name of the institution, ...


In [15]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [16]:
print(text_tables)

employees: Employee information, name, department, position, etc.
salary: Salary details for each year, including bonuses and deductions.
studies: Educational studies, name of the institution, type of studies, level of degree.


In [17]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [18]:
# إنشاء الاستفسار الذي سيُرسل إلى نموذج GPT-3.5
pqt1 = prompt_question_tables.format(tables=text_tables, question="What are the salary details of employees in 2020?")

In [20]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary"]
}


In [22]:
pqt3 = prompt_question_tables.format(tables=text_tables, question="What are the salary details of employees in 2020?")


In [23]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "salary"]
}


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

in this exercise, GPT-3.5 was used to identify the necessary tables for creating an SQL query that answers user queries. Various queries were tested to evaluate the model's ability to select relevant tables based on their definitions. A table was created containing three tables with their definitions: the `employees` table, which contains information about employees such as name, department, and role; the `salary` table, which contains salary details for each year; and the `studies` table, which contains information about educational studies, such as institution name, study type, and level. Different queries were created to test which tables were needed based on these definitions.
Some of the queries used to test the model included: "What are the salary details of employees in 2020?", for which the expected result was to identify the relevant tables as `employees` and `salary` since the query concerns salary details of employees in 2020. Another query was "Which employees graduated from studies in 2020?", where the expected result was identifying `employees` and `studies` as the relevant tables. A third query, "What is the total salary expenditure of all employees in 2021?", expected the model to identify `salary` and `employees` as the relevant tables for calculating the total salary expenditure.
The queries were tested repeatedly to find the correct tables, and the results were as expected. For the query "What are the salary details of employees in 2020?", the model correctly identified the `employees` and `salary` tables. Similarly, for the query "Which employees graduated from studies in 2020?", the correct tables, `employees` and `studies`, were identified. For the query "What is the total salary expenditure of all employees in 2021?", the model correctly identified `salary` and `employees` as the necessary tables.
However, some challenges arose during the tests. In some cases, when the queries were more complex or required information not available in the table definitions, the model started to "guess" or generate incorrect information. For instance, when using complex queries like "Which employees worked in the marketing department in 2020?", the model sometimes selected irrelevant tables. Additionally, when queries were vague or required indirect interpretation of the content, the model failed to correctly identify the necessary tables.
From this exercise, I learned the importance of clear and precise table definitions. When the table definitions were well-defined, the model performed better in selecting the relevant tables. On the other hand, when the definitions were unclear or vague, the model's responses were less accurate. Additionally, the model occasionally predicted additional tables that were not needed for a query, such as including `studies` in queries related to salaries only.
